<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas Chat - Create plugin with commands

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-10-19 (Created: 2023-10-17)

**Description:** This notebook creates a Naas Chat plugin with commands.

**References:**
- [Naas Chat Documentation](https://site.naas.ai/docs/platform/aI-powered-chat)
- [Naas Chat Plugin driver](https://github.com/jupyter-naas/drivers/blob/main/naas_drivers/tools/naas_chat_plugin.py)

## Input

### Import libraries

In [1]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Mandatory**
- `name`: The name of the plugin.
- `system_prompt`: Plugin system prompt with the commands they can use.
- `model`: This variable holds the openai model.
- `temperature`: This variable holds the temperate value.
- `description`: This variable holds a string that describes the plugin.
- `avatar`: This variable holds a URL to an image to be displayed in your plugin as avatar.
- `input_dir`: This variable holds a relative path to the directory where input notebooks with commands will be stored.

**Optional**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `output_path`: This variable holds the full path of the plugin.

In [2]:
# Inputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "GROWTH"
image_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())

name = "🚀 Growth Assistant"
model = "gpt-3.5-turbo-16k"
temperature = 0.5
description = "This plugin is used to setup the Growth Engine"
avatar = ""
input_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "growth-engine")

# Outputs
output_dir = os.path.join(naas_data_product.ROOT_PATH, "outputs", "plugins")
os.makedirs(output_dir, exist_ok=True) # Create dirs
output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")

## Model

### Get assets

In [3]:
image_input = os.path.join(image_dir, "growth.png")
image_url = naas.asset.find(image_input)
print("Image URL:", image_url)

Image URL: https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/7c7d0fea1ecb10b0f4486b6473588245f972f98506e8d96e40e21d88f425.png


### Get data from Google Sheets spreadsheet

In [4]:
# Get data
data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on this week and last week
tw = date.today().strftime("W%W-%Y")
data = data[data["SCENARIO"].isin([tw])]

# Display result
print("Rows:", len(data))
data.head(1)

Rows: 33


,ENTITY,SCENARIO,PLATFORM,FULLNAME,COHORT,OCCUPATION,PROFILE_URL,INTERACTION_SCORE,INTERACTIONS,LAST_INTERACTION_DATE
0,Jérémy Ravenel,W48-2023,LinkedIn,Jonathan La Cruz,W44-2023,Information Technology & Business intelligence...,https://www.linkedin.com/in/ACoAAAlYSwQBS9GBiC...,4,"Comment 'Congratulations on completing the ""da...",Tue. 28 Nov.


### Create system prompt

In [5]:
system_prompt = f"""
Act as a Growth Assistant who has access to a list of interactions from content that enable the user to get qualified contacts.
Your role is to manage and optimize the list of people who interacted on the content, ensuring to extract only the most qualified contacts to feed the sales representative.
You will need to analyze data to define preferences on how to touch the contact needs and interests.
Your ultimate goal is to create engaging and relevant list of qualified leads, contributing to the overall success of the sales & marketing strategy.
The first message should be about presenting yourself with a maximum of 3 bullet points and displaying an image inside the markdown of the chat: ![{image_url}].
You must ALWAYS show the image in the first message.
Then, wait for the first answer from the user, and then start with the first high-level analysis. 
Here is the data from the content analytics that you should focus on: {data.to_string}
"""

### Create commands

In [6]:
commands = create_command(input_dir)

👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



...

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
[
    {
        "name": "HubSpot_Connect",
        "avatar": "https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png",
        "action": {
            "request_type": "POST",
            "url": "https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/notebook/cebf493497f430fc99ba439d1407e11f4948422f06db61f0c8f4d0600c73",
            "payload": {
                "hs_access_token": {
                    "type": "str",
                    "description": "This variable stores an access token used for accessing the HubSpot API. This value will be stored under the secret 'HS_ACCESS_TOKEN'.",
                    "default": "None"
                }
            }
        },
        "description": "This command integrates your HubSpot access token into the Naas Lab secret manager."
    },
    {
        "name": "LinkedIn_Connect",
        "avatar": "https://raw.githubusercontent

## Output

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [7]:
# Create plugin
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_path=output_path,
    description=description,
    avatar=avatar,
    commands=commands
)

✅ System prompt tokens count OK: 3248 (limit: 20% -> 3277)
💾 Plugin successfully saved. You can use it in your Naas Chat with: /home/ftp/abi/outputs/plugins/🚀_growth_assistant.json


### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the last cell output.

In [8]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})
print(plugin_url)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/e65caf57ac5c44002f2c6b68d4ed15d240daa9367df5eee06aee1792b6a8


### Create new chat
NB: You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [9]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")

[Create New Chat](https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/e65caf57ac5c44002f2c6b68d4ed15d240daa9367df5eee06aee1792b6a8)